# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession
        .builder
        .appName("PythonFlights")
        .getOrCreate())

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("C:/Users/xabier.jimenez/Documents/EjerNavidadAdrian/Teoria y ejercicios/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [32]:
averageDelayOriginDestDF = cleanFlightsDF.groupby(F.col("Origin"),F.col("Dest"))\
                .agg(F.mean("ArrDelay").alias("avgDelay"), F.count("Origin"))\
                .sort(F.col("avgDelay").desc()) 

averageDelayOriginDestDF.show()

+------+----+-----------------+-------------+
|Origin|Dest|         avgDelay|count(Origin)|
+------+----+-----------------+-------------+
|   RDM| MFR|           1347.0|            2|
|   MDT| HPN|            798.0|            1|
|   ORD| GTF|            212.0|            1|
|   ICT| DAY|            210.0|            1|
|   ELM| ATL|            169.0|            2|
|   DSM| PIA|            168.0|            1|
|   MSP| LEX|            153.0|            2|
|   YNG| PIE|            141.0|            1|
|   CMH| HOU|            120.0|            1|
|   HRL| DAL|            111.0|            1|
|   PIE| YNG|            104.0|            1|
|   PPG| HNL|96.51851851851852|           27|
|   HNL| PPG|96.22222222222223|           27|
|   AVP| SFB|             93.0|            1|
|   CPR| LAS|             85.0|            1|
|   LAS| CPR|             82.0|            1|
|   MSP| PVD|             74.0|            1|
|   TUL| OKC|             69.0|            1|
|   ORF| BOS|67.96825396825396|   

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [18]:
aux = cleanFlightsDF.groupby(F.col("ArrDelayCat")).count()

aux.show()

+-----------+-------+
|ArrDelayCat|  count|
+-----------+-------+
|     Slight| 220431|
|       None|1472369|
|       Huge| 154961|
+-----------+-------+



In [21]:
aux = cleanFlightsDF.groupby("Origin", "Dest").pivot("ArrDelayCat").count()

aux.show()

+------+----+----+----+------+
|Origin|Dest|Huge|None|Slight|
+------+----+----+----+------+
|   MCI| MKE|   4| 133|     9|
|   TPA| ACY|   3|  84|     3|
|   PBI| DCA|  31| 283|    40|
|   DSM| EWR|   6|  71|    11|
|   MDW| MEM|  16| 130|    33|
|   ORD| PDX|  17| 379|    54|
|   SMF| BUR|  43| 529|    90|
|   STS| PHX|   8|  77|    13|
|   MCI| IAH|  35| 351|    49|
|   FSD| ATL|   6|  57|     8|
|   PHL| MCO| 129| 976|   194|
|   ATL| GSP|  37| 803|    77|
|   SJC| LIH|null|  57|     2|
|   DSM| MCO|   1|  28|     9|
|   PBG| PGD|null|  14|     3|
|   LBB| DEN|  17| 133|    18|
|   SNA| PHX|  49| 668|   221|
|   PIE| AVP|null|   1|  null|
|   LAS| LIT|   5|  71|    14|
|   BQN| MCO|   7|  62|    20|
+------+----+----+----+------+
only showing top 20 rows



<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

In [24]:
aux = cleanFlightsDF.groupby(F.col("DayOfWeek"))\
                .agg(F.mean("ArrDelay").alias("avgArrDelay"), F.mean("DepDelay").alias("avgDepDelay"))\
                .sort(F.col("avgArrDelay").desc()) 

aux.show()

+---------+------------------+------------------+
|DayOfWeek|       avgArrDelay|       avgDepDelay|
+---------+------------------+------------------+
|        5|4.9736884726946125|10.629108825829475|
|        1| 4.650733550708921|  9.92399102152194|
|        7| 3.376523577923451|  9.55569329369848|
|        4| 3.282345946624618|  9.07592391404423|
|        3|3.0552590860507562|  8.66911996797364|
|        2|2.3225243088655865| 8.165943881432788|
|        6| -0.73224717730458| 6.463834181004942|
+---------+------------------+------------------+



In [31]:
aux = cleanFlightsDF.groupby("Origin").pivot("DayOfWeek")\
                .agg(F.mean("ArrDelay").alias("avgArrDelay"))\
                .sort(F.col("Origin").desc()).toPandas()

aux

,Origin,1,2,3,4,5,6,7
0,YUM,-9.019231,-10.608696,-0.384615,-5.333333,-9.769231,-11.924528,-11.395833
1,YNG,9.000000,NaN,NaN,141.000000,NaN,NaN,NaN
2,YKM,-1.022222,-0.790698,6.111111,-0.187500,1.595745,0.153846,1.279070
3,YAK,-8.809524,-14.280000,-13.120000,6.840000,-18.576923,-17.807692,-12.565217
4,XNA,11.036585,10.719917,7.234568,3.889749,4.563601,-1.344961,6.665860
...,...,...,...,...,...,...,...,...
348,ABY,18.078947,4.942857,19.558824,15.472222,13.263158,40.125000,33.812500
349,ABR,13.840000,11.416667,8.346154,22.961538,14.653846,3.840000,4.750000
350,ABQ,2.146394,-0.114512,0.872054,0.125395,-1.700319,-4.079787,0.992727
351,ABI,13.591549,22.238095,1.116667,1.434783,5.100000,8.266667,4.307692


<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

In [79]:
averageDelayOriginDestDF = cleanFlightsDF.groupby(F.col("Origin"),F.col("Dest"))\
                .agg(F.mean("ArrDelay").alias("avgDelay"))\
                .sort(F.col("avgDelay").desc()) 

aux= averageDelayOriginDestDF.join(cleanFlightsDF,\
                                   on =(averageDelayOriginDestDF["Origin"] == cleanFlightsDF["Origin"]) &\
                                        (averageDelayOriginDestDF["Dest"] == cleanFlightsDF["Dest"]))\
                        .select(averageDelayOriginDestDF["Origin"],\
                        averageDelayOriginDestDF["Dest"], "DayOfWeek", "ArrDelay", "avgDelay").show()
aux

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.908791208791209|
|   ATL| GSP|        3|    -3.0|-1.908791208791209|
|   ATL| GSP|        3|    47.0|-1.908791208791209|
|   ATL| GSP|        4|    null|-1.908791208791209|
|   ATL| GSP|        4|    -6.0|-1.908791208791209|
|   ATL| GSP|        4|    38.0|-1.908791208791209|
|   ATL| GSP|        1|   -13.0|-1.908791208791209|
|   ATL| GSP|        1|   -15.0|-1.908791208791209|
|   ATL| GSP|        1|    10.0|-1.908791208791209|
|   ATL| GSP|        5|    -6.0|-1.908791208791209|
|   ATL| GSP|        5|    -3.0|-1.908791208791209|
|   ATL| GSP|        5|   -14.0|-1.908791208791209|
|   ATL| GSP|        1|    10.0|-1.908791208791209|
|   ATL| GSP|        1|    14.0|-1.908791208791209|
|   ATL| GSP|        1|    -3.0|-1.908791208791209|
|   ATL| GSP|        2|   -22.0|-1.908791208791209|
|   ATL| GSP

In [37]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest")\
                                         .agg(F.mean("ArrDelay").alias("avgDelay"))

joinedDF = cleanFlightsDF.join(averageDelayOriginDestDF, 
                               on = ["Origin", "Dest"], 
                               how = "left_outer")\
                         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgDelay")

joinedDF.show()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.908791208791209|
|   ATL| GSP|        3|    -3.0|-1.908791208791209|
|   ATL| GSP|        3|    47.0|-1.908791208791209|
|   ATL| GSP|        4|    null|-1.908791208791209|
|   ATL| GSP|        4|    -6.0|-1.908791208791209|
|   ATL| GSP|        4|    38.0|-1.908791208791209|
|   ATL| GSP|        1|   -13.0|-1.908791208791209|
|   ATL| GSP|        1|   -15.0|-1.908791208791209|
|   ATL| GSP|        1|    10.0|-1.908791208791209|
|   ATL| GSP|        5|    -6.0|-1.908791208791209|
|   ATL| GSP|        5|    -3.0|-1.908791208791209|
|   ATL| GSP|        5|   -14.0|-1.908791208791209|
|   ATL| GSP|        1|    10.0|-1.908791208791209|
|   ATL| GSP|        1|    14.0|-1.908791208791209|
|   ATL| GSP|        1|    -3.0|-1.908791208791209|
|   ATL| GSP|        2|   -22.0|-1.908791208791209|
|   ATL| GSP

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

In [44]:

aux.show(10)

+------+----+---------+--------+------------------+------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|belowAverage|
+------+----+---------+--------+------------------+------------+
|   ATL| GSP|        3|    -6.0|-1.908791208791209|        true|
|   ATL| GSP|        3|    -3.0|-1.908791208791209|        true|
|   ATL| GSP|        3|    47.0|-1.908791208791209|       false|
|   ATL| GSP|        4|    null|-1.908791208791209|        null|
|   ATL| GSP|        4|    -6.0|-1.908791208791209|        true|
|   ATL| GSP|        4|    38.0|-1.908791208791209|       false|
|   ATL| GSP|        1|   -13.0|-1.908791208791209|        true|
|   ATL| GSP|        1|   -15.0|-1.908791208791209|        true|
|   ATL| GSP|        1|    10.0|-1.908791208791209|       false|
|   ATL| GSP|        5|    -6.0|-1.908791208791209|        true|
+------+----+---------+--------+------------------+------------+
only showing top 10 rows



**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [45]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [52]:
joinedFlightsDF = cleanFlightsDF.join(airportsDF, on = cleanFlightsDF.Origin == airportsDF.IATA)

# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.filter(~(F.col("FullName").isNull()))\
               .select("Origin", "Dest", "IATA", "FullName", "Year").show()

+------+----+----+--------------------+----+
|Origin|Dest|IATA|            FullName|Year|
+------+----+----+--------------------+----+
|   SEA| JFK| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| JFK| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| JFK| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| JFK| SEA|Seattle-Tacoma In...|1949|
|   SEA| LGB| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| BOS| SEA|Seattle-Tacoma In...|1949|
|   SEA| L

## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [70]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión

cleanFlightsDF2 = cleanFlightsDF.withColumn("Distance", F.col("Distance").cast(DoubleType()))\
                                        .filter(F.col("Distance").isNotNull())

flightsWithKm = cleanFlightsDF2.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM").distinct().show(10)

8.0
+------+----+--------+------------------+
|Origin|Dest|Distance|            DistKM|
+------+----+--------+------------------+
|   CLE| JFK|   425.0|             680.0|
|   MCO| PSE|  1179.0|            1886.4|
|   FLL| DTW|  1127.0|            1803.2|
|   LAX| MTJ|   666.0|1065.6000000000001|
|   AZA| MLI|  1288.0|            2060.8|
|   CVG| MSY|   701.0|1121.6000000000001|
|   MFR| LAX|   630.0|            1008.0|
|   PVD| PIE|  1145.0|            1832.0|
|   FLL| ORF|   781.0|1249.6000000000001|
|   FAT| SFO|   158.0|             252.8|
+------+----+--------+------------------+
only showing top 10 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [74]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daysOfWeek[dayInteger]
    
# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = F.udf(dayOfWeekToString, StringType())

# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()

+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   BQK| ATL|        4|       Thursday|
|   CVG| PHL|        3|      Wednesday|
|   DTW| DFW|        5|         Friday|
|   SEA| JFK|        2|        Tuesday|
|   JAX| JFK|        2|        Tuesday|
|   RDU| BOS|        3|      Wednesday|
|   SEA| BOS|        3|      Wednesday|
|   AUS| FLL|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   SLC| BOS|        6|       Saturday|
|   BOS| HOU|        6|       Saturday|
|   BDL| MCO|        7|         Sunday|
|   SJU| TPA|        7|         Sunday|
|   PGD| TYS|        6|       Saturday|
|   PIE| CVG|        6|       Saturday|
|   ABE| SFB|        7|         Sunday|
|   LAS| BIS|        7|         Sunday|
|   ROC| PGD|        1|         Monday|
|   EWR| CVG|        1|         Monday|
|   CVG| SFB|        1|         Monday|
+------+----+---------+---------------+
only showing top 20 rows

